In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.preprocessing
import numpy as np
from scipy import stats

df = pd.read_csv('credit_card_churn.csv')
df = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis=1)

df = df.loc[df['Card_Category'] != 'Platinum']
df = df.loc[df['Total_Ct_Chng_Q4_Q1'] <= 1.6]
df = df.loc[df['Months_Inactive_12_mon'] >= 1]
df = df.loc[df['Months_Inactive_12_mon'] < 6]
df = df.loc[df['Customer_Age'] <= 66]
df = df.loc[df['Total_Amt_Chng_Q4_Q1'] <= 1.5]
df['Credit_Limit'] = np.log10(df['Credit_Limit'])
fitted_data, fitted_lambda = stats.boxcox(df['Avg_Open_To_Buy'])
df['Avg_Open_To_Buy'] = fitted_data
df['Total_Trans_Amt'] = np.log10(df['Total_Trans_Amt'])
df['Avg_Utilization_Ratio'] = (df['Avg_Utilization_Ratio'])**(1/2)

Data Exploration
ข้อมุลไม่มีช่องไหนที่หายไป
ข้อมูลประกอบไปด้วย feature 21 ตัวดังนี้
- Clientnum: หมายเลขที่ใช้ระบุลูกค้า
- Customer Age: อายุ
- Gender: เพศ
- Dependent count: จำนวนบุตรที่อยู่ในการอุปการะ
- Education level: การศึกษา
- Martial Status: สถานภาพ
- Income Category: หมวดหมู่รายได้ประจำปี
- Card Category: ชนิดของบัตรที่ถือ
- Months on book: **
- Total Relationship Count: จำนวน product ที่ถืออยู่
- Month inactive last 12 months: จำนวนเดือนที่ไม่ได้ใช้งาน
- Contact Count last 12 months: จำนวนการติดต่อ
- Credit Limit: จำกัดวงเงินในบัตรเครดิต
- Total Revolving Balance: ยอดเงินหมุนเวียนในบัตรเครดิต
- Average Open to buy: **
- Change in Transaction Amount (Q4 over Q1): ยอดการเปลี่ยนแปลงธุรกรรม
- Total Transaction Amount (Last 12 months): ยอดรวมธุรกรรม
- Total transaction count last 12 months: จำนวนธุรกรรม
- Change in Transaction Count (Q4 over Q1): จำนวนครั้งการเปลี่ยนแปลงธุรกรรม
- Average Card Utilization Ratio: อัตราการใช้บัตรเฉลี่ย
- Attrition_Flag: เธอจะอยู่หรือเธอจะไป

In [ ]:
print(df.isnull().sum())

##### Clientnum
- รหัสลูกค้าเป็นข้อมูลที่ดูไม่มีความสัมพันธุ์อะไรกับข้อมูลอื่นๆที่สื่อถึงการสูญเสียลูกค้าแต่อาจจะมีความสัมพันธุ์กับการระบุตัวตนของผู้ใช้งานหรือประเภทบัตรได้
- เมื่อ plot ดูความสัมพันธุ์แล้วพบว่าไม่มีความสัมพันธุ์ใดๆ จึงทำการตัดข้อมูล Clientnum ออก

In [ ]:
df1 = df[['CLIENTNUM', 'Customer_Age', 'Attrition_Flag']]
card_cat = df['Card_Category']
label_encoder = sklearn.preprocessing.LabelEncoder()
card_cat = label_encoder.fit_transform(card_cat)
df1['Card_Category'] = card_cat
#df1.loc[:, 'Card_Category'] = card_cat
plt.figure(figsize = (8,4))
sns.pairplot(df1, hue="Attrition_Flag")


##### Attrition_Flag
    - ข้อมูลมีความ imbalance โดยที่ Existing Customer มากกว่า Attried Customer จำนวนมาก
    - ข้อมูล Total_Trans_Amt, Total_Trans_Ct และ Total_Ct_Chng_Q4_Q1 ดูมีความสัมพันธุ์กับ output เนื่องจากสังเกตุได้ถึงการแยกกลุ่มของข้อมูลเหล่านี้

In [ ]:
df = df.drop(['CLIENTNUM'], axis=1)
cols = df.columns
num_data = list(df._get_numeric_data().columns)
cat_data = list(set(cols) - set(num_data))
num_data.remove('Dependent_count')
num_data.remove('Total_Relationship_Count')
num_data.remove('Months_Inactive_12_mon')
num_data.remove('Contacts_Count_12_mon')
cat_data.append('Dependent_count')
cat_data.append('Total_Relationship_Count')
cat_data.append('Months_Inactive_12_mon')
cat_data.append('Contacts_Count_12_mon')
num_data_plot = num_data
cat_data_plot = cat_data


In [ ]:
plt.figure(figsize = (6,3))
sns.countplot(data=df, x="Attrition_Flag")
df['Attrition_Flag'].value_counts()

In [ ]:
for i in num_data_plot:
    sns.boxplot(data=df, x='Attrition_Flag', y= i)
    plt.show()
cat_data_plot.remove('Attrition_Flag')
for j in cat_data_plot:
    sns.countplot(data=df, x=j, hue="Attrition_Flag")
    plt.show()

##### Customer Age
    - ข้อมูลอายุมีการกระจายเกือบจะเป็น normal distribution โดยลูกค้าส่วนมากมีอายุช่วง 40 - 50 ปี มีความแปรปรวนเท่ากับ 64
    - ช่วงอายุไม่มีความสัมพันธุ์กับการสูญเสียลูกค้าทั้ง Existing customer และ Attrited customer มีอายุเฉลี่ยประมาณ 45 เท่ากัน
    - ช่วงอายุที่มากกว่า 66 อาจพิจารณาเป็น outlier
    - ข้อมูลอายุมีความสัมพันธุ์กับฟีเจอร์ month on book แต่ไม่เห็นถึงความสัมพันธุ์ต่อ output จึงอาจพิจารณาตัดข้อมูลใดข้อมูลหนึ่งออก
    - ข้อมูล Customer_Age เมื่อ plot กับ Total_Trans_Amt เห็นถึงการแบ่งกลุ่มของ output ในช่วงที่ Total_Trans_Amt มากกว่า 12000

In [ ]:
plt.figure(figsize = (6,3))
sns.boxplot(data=df, x='Attrition_Flag', y='Customer_Age')
plt.figure(figsize = (6,3))
sns.histplot(data=df, x="Customer_Age", hue="Attrition_Flag", multiple="stack", kde=True)
df['Customer_Age'].describe()

In [ ]:
num_data_plot.remove('Customer_Age')
for i in num_data_plot:
    sns.jointplot(data=df, x=i, y="Customer_Age", hue="Attrition_Flag")
    plt.show()
for j in cat_data_plot:
    sns.boxplot(data=df, x=j, y='Customer_Age', hue = 'Attrition_Flag')
    plt.show()

##### Gender
    - จำนวนประชากรเพศชายกับหญิงมีจำนวนห่างกันเพียง 589 คน
    - ข้อมูลเพศของลูกค้าไม่มีความสัมพันธุ์กับ output ของข้อมูล และไม่มีความสัมพันธุ์ใดๆกับข้อมูลอื่นๆ
    - เพศหญิงมี credit limit และ Avg_open_to_buy ต่ำกว่าเพศชายอย่างเห็นได้ชัด
    - ในข้อมูลของ income กลุ่มที่มีรายได้ต่ำกว่า 40000 และรายได้ที่ไม่ทราบส่วนใหญ่เป็นเพศหญิง
    - กลุ่มรายได้ 60k-80k, 80k-120k, 120k+ ส่วนใหญ่เป็นเพศชาย
    - ข้อมูล Total_Trans_Amt ในเพศหญิงมีการแบ่งกลุ่มโดย Existing Customer มี Total_Trans_Amt อยู่ในช่วง 2500 – 500 แต่ Attrited Customer อยู่ในช่วง 2500 +- เล็กน้อยแต่ในเพศชายไม่เห็นการแบ่งกลุ่มโดยมี mean ที่ใกล้เคียงกัน
    - ข้อมูล Total_Trans_Ct ในเพศหญิงมีการแบ่งกลุ่มโดย Existing Customer มี Total_Trans_Ct อยู่ในช่วง 60-80 แต่ Attrited Customer อยู่ในช่วง 40-50 แต่ในเพศชายไม่เห็นการแบ่งกลุ่มอย่างชัดเจนแต่สังเกตุเห็นได้ว่ากลุ่ม Attrited Customer จะมีช่วง Totl_Trans_Ct ต่ำกว่ากลุ่ม Existing Customer


In [ ]:
plt.figure(figsize = (6,3))
sns.countplot(data=df, x="Gender", hue="Attrition_Flag")
df['Gender'].value_counts()

In [ ]:
for i in num_data_plot:
    sns.boxplot(data=df, x='Gender', y= i, hue = 'Attrition_Flag')
    plt.show()
cat_data_plot.remove('Gender')
for j in cat_data_plot:
    sns.countplot(data=df, x=j, hue="Gender")
    plt.show()

##### Dependent Count
    - ลูกค้าส่วนมากมีบุตร 2-3 คน
    - จำนวนบุตรไม่มีความสัมพันธุ์ใดๆกับข้อมูลอื่นๆจึงพิจารณาที่จะตัดทิ้งได้
    - เมื่อ plot total_trans_ct กับ dependent count ทำให้เห็นการจัดกลุ่มของการสูญเสียลูกค้าได้ดีมีซ้อนบ้างเล็กน้อย
    - แต่การดู boxplot ทำให้เห็นได้ชัดเจนว่าข้อมูล total revolving balance ดูมีความสัมพันธุ์กับการสูญเสียลูกค้า

In [ ]:
sns.histplot(data=df, x="Dependent_count", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
for i in num_data_plot:
    sns.boxplot(data=df, x='Dependent_count', y= i, hue = 'Attrition_Flag')
    plt.show()
cat_data_plot.remove('Dependent_count')
for j in cat_data_plot:
    sns.countplot(data=df, x=j, hue="Dependent_count")
    plt.show()

##### Education Level
    - ลูกค้าส่วนมากเป็นลูกค้าที่เรียนจบแล้ว
    - จากการวิเคราะฟีเจอร์ Dependent count ซึ่งพบว่า total_trans_ct เมื่อ plot กับ Education Level แล้วพบว่าเกิดการแบ่งกลุ่มของการสูญเสียลูกค้าได้อย่างชัดเจน

In [ ]:
plt.figure(figsize = (8,4))
sns.countplot(data=df, x="Education_Level", hue="Attrition_Flag")
df['Education_Level'].value_counts()

In [ ]:
for i in num_data_plot:
    sns.boxplot(data=df, x='Education_Level', y= i, hue = 'Attrition_Flag')
    plt.show()
    
cat_data_plot.remove('Education_Level')
for j in cat_data_plot:
    sns.countplot(data=df, x=j, hue="Education_Level")
    plt.show()

##### Marital Status
    - สถานภาพไม่มีความสัมพันธุ์ใดๆกับข้อมูลอื่นๆ

In [ ]:
plt.figure(figsize = (8,4))
sns.countplot(data=df, x="Marital_Status", hue="Attrition_Flag")
df['Marital_Status'].value_counts()

In [ ]:
for i in num_data_plot:
    sns.boxplot(data=df, x='Marital_Status', y= i, hue = 'Attrition_Flag')
    plt.show()
cat_data_plot.remove('Marital_Status')
for j in cat_data_plot:
    sns.countplot(data=df, x=j, hue="Marital_Status")
    plt.show()

##### Income Category
    - ลูกค้าส่วนใหญ่มีรายได้ต่ำกว่า 40000
    - คนที่มีรายได้มากกว่า 120k มี credit limit และ avg_open_to_buy สูงกว่ากลุ่มอื่นๆ

In [ ]:
plt.figure(figsize = (8,4))
sns.countplot(data=df, x="Income_Category", hue="Attrition_Flag")
df['Income_Category'].value_counts()

In [ ]:
for i in num_data_plot:
    sns.boxplot(data=df, x='Income_Category', y= i, hue = 'Attrition_Flag')
    plt.show()
cat_data_plot.remove('Income_Category')
for j in cat_data_plot:
    sns.countplot(data=df, x=j, hue="Income_Category")
    plt.show()

##### Card Category
    - ผู้ใช้งานส่วนมากเป็นผู้ใช้งานระดับ Blue
    - ผู้ใช้งานระดับ Blue มี avg_open_to_buy และ credit limit ต่ำกว่ากลุ่มอื่นๆแต่เนื่องจากข้อมูลที่ไม่ใช่ระดับ Blue น้อยมากจึงพิจารณาที่จะตัดข้อมูลนี้ทิ้ง

In [ ]:
plt.figure(figsize = (8,4))
sns.countplot(data=df, x="Card_Category", hue="Attrition_Flag")
df['Card_Category'].value_counts()

In [ ]:
for i in num_data_plot:
    sns.boxplot(data=df, x='Card_Category', y= i, hue = 'Attrition_Flag')
    plt.show()
cat_data_plot.remove('Card_Category')
for j in cat_data_plot:
    sns.countplot(data=df, x=j, hue="Card_Category")
    plt.show()

##### Months_on_book
    - จำนวนมากสุด 36 คือ 2463 คน
    - เมื่อ plot ค่า Months_on_book กับ Total_Trans_Amt พบว่าช่วงที่ Total_Trans_Amt มากกว่า 12000 เห็นกลุ่มของ Existing Customer ชัดเจน แต่ช่วงที่น้อยกว่า 12000 ยังคงเห็นการปนกันของ Existing Customer และ Attrited Customer แต่การที่เห็นการแบ่งกลุ่มชัดเจนเนื่องมาจากตัวข้อมูลของ Total_Trans_Amt ที่มีการแบ่งกลุ่ม output ในตัวของข้อมูลอยู่แล้ว

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Months_on_book", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
num_data_plot.remove('Months_on_book')
for i in num_data_plot:
    sns.jointplot(data=df, x=i, y="Months_on_book", hue="Attrition_Flag")
    plt.show()
for j in cat_data_plot:
    sns.boxplot(data=df, x=j, y='Months_on_book', hue = 'Attrition_Flag')
    plt.show()

##### Total_Relationship_Count
    - ข้อมูลไม่มีความสัมพันธุ์โดยตรงกับ Attrition Flag
    - แต่เมื่อ plot กับ total_trans_amt พบว่าเกิดการแบ่งแยกของ Attriton Flag แต่ยังมีทับซ้อนกันบางส่วน

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Total_Relationship_Count", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
for i in num_data_plot:
    sns.boxplot(data=df, x='Total_Relationship_Count', y= i, hue = 'Attrition_Flag')
    plt.show()
cat_data_plot.remove('Total_Relationship_Count')
for j in cat_data_plot:
    sns.countplot(data=df, x=j, hue="Total_Relationship_Count")
    plt.show()

In [ ]:
sns.pairplot(df, x_vars= 'Total_Trans_Amt', y_vars=["Total_Relationship_Count"], hue = 'Attrition_Flag')
plt.show()

##### Months Inactive 12_mon
    - ข้อมูลมีความสัมพันธุ์กับ total revolving balance และ total_trans_ct

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Months_Inactive_12_mon", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
for i in num_data_plot:
    sns.boxplot(data=df, x='Months_Inactive_12_mon', y= i, hue = 'Attrition_Flag')
    plt.show()
cat_data_plot.remove('Months_Inactive_12_mon')
for j in cat_data_plot:
    sns.countplot(data=df, x=j, hue="Months_Inactive_12_mon")
    plt.show()

##### Contacts_Count_12_mon
    - contacts_count_12_mon จำนวน 6 ครั้ง พบว่าเป็นลูกค้าที่เสียไปทั้งหมด แต่เป็นเพียงข้อมูลเล็กน้อยของข้อมูลทั้งหมด

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Contacts_Count_12_mon", hue="Attrition_Flag", multiple="stack", kde=True)
df['Contacts_Count_12_mon'].value_counts()

In [ ]:
for i in num_data_plot:
    sns.boxplot(data=df, x='Contacts_Count_12_mon', y= i, hue = 'Attrition_Flag')
    plt.show()
cat_data_plot.remove('Contacts_Count_12_mon')

##### Credit_Limit
    - จำนวน credit limit มีลักษณะ right skewed จึงทำการ correct skewed data ด้วยวิธีการ Box-cox แต่ส่งผลให้รูปแบบความสัมพันธุ์ของ Credit_Limit กับ Avg_open_to_buy เปลี่ยนจาก linear เป็น parabola
    - ข้อมูลมีความสัมพันธุ์กับ Avg_Utilization_Ratio เมื่อ Credit_Limit เพิ่มมากขึ้น Avg_Utilization_Ratio ก็จะต่ำลงอย่างเห็นได้ชัด

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Credit_Limit", hue="Attrition_Flag", multiple="stack", kde=True)

from scipy import stats

df1 = df
df1['Credit_Limit'], fitted_lambda  = stats.boxcox(df1['Credit_Limit'].values)
plt.figure(figsize = (8,4))
sns.histplot(data=df1, x="Credit_Limit", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
num_data_plot.remove('Credit_Limit')
for i in num_data_plot:
    sns.jointplot(data=df, x="Credit_Limit", y=i, hue="Attrition_Flag")
    plt.show()


##### Total_Revolving_Bal
    - ข้อมูลมีความสัมพันธุ์กับข้อมูล Avg_Utilization_Ratio แต่มีความแปรปรวนมาก

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Total_Revolving_Bal", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
num_data_plot.remove('Total_Revolving_Bal')
for i in num_data_plot:
    sns.jointplot(data=df, x="Total_Revolving_Bal", y=i, hue="Attrition_Flag")
    plt.show()


##### Avg_Open_To_Buy
    - ข้อมูลมีลักษณะ Right Skewed
    - ข้อมูล Avg_open_To_Buy มีความสัมพันธุ์กับ Avg_Utilzation_Ratio เมื่อค่า Avg_open_To_Buy มากขึ้น Avg_Utilzation_Ratio ลดลงอย่างเห็นได้ชัดจึงอาจพิจารณาตัดข้อมูลใดข้อมูลนึงออก


In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Avg_Open_To_Buy", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
num_data_plot.remove('Avg_Open_To_Buy')
for i in num_data_plot:
    sns.jointplot(data=df, x="Avg_Open_To_Buy", y=i, hue="Attrition_Flag")
    plt.show()


##### Total_Amt_Chng_Q4_Q1
    - จากกราฟ histrogram จะสังเกตุได้ว่าเมื่อ Total_Amt_Chng_Q4_Q1 มากกว่า 1 ข้อมูลที่เหลือจะเป็น Existing Customer แต่เป็นเพียงข้อมูลส่วนน้อย
    - Total_Amt_Chng_Q4_Q1 มีความสัมพันธุ์กับ Total_Trans_Amt มองเห็นการแบ่งกลุ่มอย่างชัดเจนเมื่อ Total_Trans_Amt มากกว่า 12000 แต่ยังคงมีกลุ่มที่ซ้อนทับกันอยู่ในช่วงที่ต่ำกว่า 12000

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Total_Amt_Chng_Q4_Q1", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
num_data_plot.remove('Total_Amt_Chng_Q4_Q1')
for i in num_data_plot:
    sns.jointplot(data=df, x="Total_Amt_Chng_Q4_Q1", y=i, hue="Attrition_Flag")
    plt.show()


##### Total_Trans_Amt
    - จากกราฟการกระจายตัวของข้อมูลพบว่าเมื่อ Total_Trans_Amt มีค่ามากกว่า 10000 ลูกค้าส่วนใหญ่เป็น Attrited Customer
    - ความสัมพันธุ์ระหว่าง Total_Trans_Amt และ Total_Trans_Ct มีการแบ่งกลุ่ม Output
    - ความสัมพันธุ์ระหว่าง Total_Trans_Amt และ Total_Ct_Chng_Q4_Q1 มีการแบ่งกลุ่ม Output
    - ความสัมพันธุ์ระหว่าง Total_Trans_Amt และ Avg_Utilization_ratio มีการแบ่งกลุ่ม Output

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Total_Trans_Amt", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
num_data_plot.remove('Total_Trans_Amt')
for i in num_data_plot:
    sns.jointplot(data=df, x="Total_Trans_Amt", y=i, hue="Attrition_Flag")
    plt.show()


##### Total_Trans_Ct

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Total_Trans_Ct", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
num_data_plot.remove('Total_Trans_Ct')
for i in num_data_plot:
    sns.jointplot(data=df, x="Total_Trans_Ct", y=i, hue="Attrition_Flag")
    plt.show()


##### Total_Ct_Chng_Q4_Q1

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Total_Ct_Chng_Q4_Q1", hue="Attrition_Flag", multiple="stack", kde=True)

In [ ]:
num_data_plot.remove('Total_Ct_Chng_Q4_Q1')
for i in num_data_plot:
    sns.jointplot(data=df, x="Total_Ct_Chng_Q4_Q1", y=i, hue="Attrition_Flag")
    plt.show()


##### Avg_Utilization_Ratio
    - ข้อมูลมีลักษณะแบบ right skewed

In [ ]:
plt.figure(figsize = (8,4))
sns.histplot(data=df, x="Avg_Utilization_Ratio", hue="Attrition_Flag", multiple="stack", kde=True)

##### Preprocessing

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.preprocessing
import numpy as np

df = pd.read_csv('credit_card_churn.csv')
df = df.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'], axis=1)
#df1 = df[['Education_Level', 'Months_Inactive_12_mon', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Attrition_Flag']]

In [ ]:
df1.head()

In [ ]:
df1 = df1.loc[df1['Total_Ct_Chng_Q4_Q1'] <= 1.6]
df1 = df1.loc[df1['Months_Inactive_12_mon'] >= 1]
df1 = df1.loc[df1['Months_Inactive_12_mon'] < 6]

In [ ]:
x = df1.iloc[:,:-1]
y = df1.iloc[:,-1]
#x = pd.get_dummies(x)
#y = y.replace(['Existing Customer', 'Attrited Customer'], [1, 0])

In [ ]:
x = pd.get_dummies(x)
x.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

x = x.values
y = y.values
scaler = MinMaxScaler()
x_scaled = scaler.fit_transform(x)
x_train,x_test,y_train,y_test = train_test_split(x_scaled, y, test_size = 0.1, random_state = 1)


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

parameters = {'C':[0.01, 0.1, 1, 10, 100],
                'solver':('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')}
logisReg = LogisticRegression()
clf = GridSearchCV(logisReg, parameters, scoring='f1')
clf.fit(x_train, y_train)

In [ ]:
print(clf.best_params_)
y_pred = clf.predict(x_test)
print(classification_report(y_test, y_pred))